In [ ]:
from peft import PeftModel, PeftConfig
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm
import evaluate
import gradio as gr
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu



In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

drive_path = '/content/drive/MyDrive/Thesis/model'


In [ ]:
peft_model_id = "/content/drive/MyDrive/Thesis/lora-mt5-xl/checkpoint-2500"

In [ ]:
config = PeftConfig.from_pretrained(peft_model_id)

In [ ]:
config.base_model_name_or_path

'google/mt5-xl'

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(drive_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(drive_path)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 2048)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 2048)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear8bitLt(
                    (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): Param

In [ ]:
metric = evaluate.load("rouge")

In [ ]:
test_dataset = pd.read_csv('/content/drive/MyDrive/Thesis/humaneval/humanevaldataset.csv')

In [ ]:
def evaluate_peft_model(sample, max_target_length=50):
    outputs = model.generate(
        input_ids=sample["input_ids"].unsqueeze(0).cuda(),
        max_length=max_target_length,
        num_beams=4,
        early_stopping=True,
        num_return_sequences=1  
    )
    prediction = tokenizer.decode(outputs[0].cpu().numpy(), skip_special_tokens=True)
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    return prediction, labels

In [ ]:
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append([l])

100%|██████████| 50/50 [04:22<00:00,  5.25s/it]


In [ ]:
smoother = SmoothingFunction().method4
bleu_score = corpus_bleu(references, predictions, smoothing_function=smoother, weights=(0.25, 0.25, 0.25, 0.25), auto_reweigh=True)
print(f"Adjusted BLEU score: {bleu_score * 100:.2f}%")

Adjusted BLEU score: 18.70%


In [ ]:
rouge = metric.compute(predictions=predictions, references=references)

In [ ]:
rouge

{'rouge1': 0.22427115564512695,
 'rouge2': 0.05013526050832552,
 'rougeL': 0.20621904584105666,
 'rougeLsum': 0.20430566211256107}

In [ ]:
example = "this ocular emergency requires intravenous antibiotics , piptolaryngology , and ophthalmologic consultation ."

word = "piptolaryngology"

In [ ]:
def generate(example, word):
  prompt = """example: {}

word: {}

provide a dictionary definition for the word based on the example.

definition: """
  input_ids = tokenizer(prompt.format(example, word), return_tensors="pt", truncation=True, max_length = 512).input_ids.cuda()
  with torch.inference_mode():
    outputs = outputs = model.generate(
        input_ids=input_ids,
        max_length=50,
        num_beams=4,
        early_stopping=True,  # Stop generation when all beam hypotheses reach the end token
        num_return_sequences=1  # Return only one generated sequence
    )
    #outputs = model.generate(input_ids=input_ids, max_new_tokens=150, do_sample=True, top_p=0.9)
  print(f"input word: {word}\n{'---'* 20}\nDefinition:")

  return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:
def generate(example, word):
  prompt = """example: {}

word: {}

provide a dictionary definition for the word based on the example.

definition: """
  input_ids = tokenizer(prompt.format(example, word), return_tensors="pt", truncation=True, max_length = 512).input_ids.cuda()
  with torch.inference_mode():
    outputs = outputs = model.generate(
        input_ids=input_ids,
        max_length=50,
        num_beams=4,
        early_stopping=True,
        num_return_sequences=1
    )
    #outputs = model.generate(input_ids=input_ids, max_new_tokens=150, do_sample=True, top_p=0.9)
  print(f"input word: {word}\n{'---'* 20}\nDefinition:")

  return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:
example = "institutions are rejecting warnings that house prices could curpe because low interest rates are likely to be held far into next year ."

word = "curpe"

In [ ]:
print(generate(example, word))

input word: curpe
------------------------------------------------------------
Definition:
( of a price ) fall to a lower level


In [ ]:
print(generate(example, word))

input word: piptolaryngology
------------------------------------------------------------
Definition:
a branch of medicine concerned with the examination of the eye, especially the eyelids.


In [ ]:
def generate_dutch(example, word):
  prompt = """voorbeeld: {}

woord: {}

geef een woordenboekdefinitie voor het woord op basis van het voorbeeld.

definitie: """
  input_ids = tokenizer(prompt.format(example, word), return_tensors="pt", truncation=True, max_length = 512).input_ids.cuda()
  with torch.inference_mode():
    outputs = outputs = model.generate(
        input_ids=input_ids,
        max_length=50,
        num_beams=4,
        early_stopping=True,  
        num_return_sequences=1
    )
  print(f"input word: {word}\n{'---'* 20}\nDefinition:")

  return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:
example = "it may also by carried by dust and long grass contaminated by infected animals so if one beast develops the condition , the others should be moved out of that pasture ."

word = "grass"

In [ ]:
print(generate(example, word))

input word: grass
------------------------------------------------------------
Definition:
( of a plant or animal ) having a shrub-like appearance


In [ ]:
def choose_and_generate(example, selected_model, word):

  if selected_model == "generate":
    return generate(example, word)
  else:
    return generate_dutch(example, word)

In [ ]:
models = ["generate", "generate_dutch"]

interface = gr.Interface(
  fn=choose_and_generate,
  inputs=[gr.Textbox("Example Text"), gr.Dropdown(choices=models, label="Model"), gr.Textbox("Word")],
  outputs="text",
  title="Text Generator",
  description="Choose a model and enter example text and a word to generate output."
)

interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2991a14f1c1aa3c8ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
